In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [3]:
import sys
sys.path.insert(0, "../code-previous")
sys.path.insert(0, "../tagger-scrap")

sys.path.insert(0, "..")

In [4]:
import os
from nltk.tag.stanford import StanfordPOSTagger
java_path = "/home/sreekumar_s/jdk1.8.0_131/bin/java"
os.environ['JAVAHOME'] = java_path


import CMUTweetTagger

In [5]:
path_to_model = "../pos-tagger-models/english-bidirectional-distsim.tagger"
path_to_jar = "../pos-tagger-models/stanford-postagger.jar"
tagger = StanfordPOSTagger(path_to_model, path_to_jar)
tagger.java_options='-mx4096m'

In [6]:
import pandas as pd
import numpy as np
import nltk
stopwords = nltk.corpus.stopwords.words('english')

import plotly

import swifter
import dask.dataframe as dd
from dask.multiprocessing import get

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set()
import mutual_information
import matplotlib.pyplot as plt
path2rawdata = '/mnt/disks/vault/wos2017-parsed/'

In [10]:
raw_data_path = "/mnt/disks/vault/analysis-data/nature-science-data-full/nature_science_journal_data.pql"
data = pd.read_pickle(raw_data_path)

data = data[(data.Journal == "NATURE") & (data.PubYear_x > 2000)]

In [11]:
len(data)

11940

In [12]:
data.head()

,ArticleID,PubYear_x,Doctypes,Title,Journal,Journal Abbreviation ISO,PubYear_y,c5
31047,000261559900061,2008,Letter,FCA does not bind abscisic acid,NATURE,Nature,2008,20.0
31048,000259090800017,2008,Letter,Doping: ignorance of basic statistics is all too common,NATURE,Nature,2008,0.0
31050,000261340000041,2008,Article,Direct control of paralysed muscles by cortical neurons,NATURE,Nature,2008,133.0
31051,000258591000018,2008,Letter,Open debate could slow flu vaccine production,NATURE,Nature,2008,0.0
31054,000253671900043,2008,Article,Cyclical DNA methylation of a transcriptionally active promoter,NATURE,Nature,2008,362.0


__Add tagging from the two parsers__

In [13]:
data['stanford_tagger_tags'] = data['Title'].swifter.apply(lambda x: tagger.tag(x.split()))

Pandas Apply: 100%|██████████| 11940/11940 [3:23:27<00:00,  1.02s/it] 


In [14]:
import pickle
data.to_pickle("/mnt/disks/vault/analysis-data/stanford_tagger_data/pos_tagged_stanford_2001.pql")

In [ ]:
#ark_tagger_outputs = CMUTweetTagger.runtagger_parse(list(data['Title']))
#data['ark_tagger_tags'] = pd.Series(ark_tagger_outputs)

In [ ]:
## https://stackoverflow.com/questions/45545110/how-do-you-parallelize-apply-on-pandas-dataframes-making-use-of-all-cores-on-o

ddata = dd.from_pandas(data, npartitions=30)

res = ddata.map_partitions(lambda df: df['Title'].apply((lambda x: tag(x.split())), axis=1)).compute(get=get)